In [1]:
!pip install sentence_transformers numpy diskcache

  Using cached sentence_transformers-3.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached transformers-4.46.2-py3-none-any.whl.metadata (44 kB)
  Using cached scikit_learn-1.5.2-cp312-cp312-macosx_12_0_arm64.whl.metadata (13 kB)
  Using cached scipy-1.14.1-cp312-cp312-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached huggingface_hub-0.26.2-py3-none-any.whl.metadata (13 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached setuptools-75.5.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Usin

In [108]:
import asyncio
import diskcache
from batched.types import AsyncCache, BatchProcessorStats, Cache, BatchProcessorCacheStats
from concurrent.futures import ThreadPoolExecutor
from typing import TypeVar
import pickle
T = TypeVar("T")
U = TypeVar("U")
import time
class AsyncDiskCacheOld(AsyncCache[T, U]):
    def __init__(self, directory: str = "/tmp/batched", n_threads: int = 16, size_limit: int | None = None, **kwargs):
        #with ensure_import("diskcache"):
        #    import diskcache
        # TODO size_limit parameter for cache size
        self._cache = diskcache.Cache(directory, size_limit=100, **kwargs)
        self._pool = ThreadPoolExecutor(max_workers=n_threads)
        self._stats = None #BatchProcessorCacheStats()

        #self._maxsize = size_limit if size_limit is not None else float('inf') #maxsize if maxsize is not None else float('inf')

    #async def get(self, key: T) -> U | None:
    #    loop = asyncio.get_running_loop()
    #    return await loop.run_in_executor(self._pool, self._cache.get, key)

    async def get(self, key: T) -> U | None:
        t0 = time.time()
        loop = asyncio.get_running_loop()
        
        # Wrap the cache.get in a function that returns both value and size
        async def get_with_stats():
            hit = await loop.run_in_executor(self._pool, self._cache.get, key)
            #self._stats.update_get(hit is not None, time.time() - t0)
            return hit

        return await get_with_stats()

    async def set(self, key: T, value: U) -> None:
        print("Setting cache")
        #self._stats.update_set(len(self._cache), self._cache.volume(), False, 1)
        loop = asyncio.get_running_loop()
        await loop.run_in_executor(self._pool, self._cache.set, key, value)

class AsyncDiskCache:
    def __init__(self, directory: str = "/tmp/batched", n_threads: int = 16, size_limit: int | None = None, **kwargs):
        print(f"\nInitializing cache:")
        print(f"Directory exists before init: {os.path.exists(directory)}")
        if os.path.exists(directory):
            print(f"Directory contents before init: {os.listdir(directory)}")
        
        kwargs['sqlite_pragma_synchronous'] = 'FULL'
        kwargs['sqlite_pragma_journal_mode'] = 'WAL'
        self._cache = diskcache.Cache(directory, size_limit=10000, **kwargs)
        self._pool = ThreadPoolExecutor(max_workers=n_threads)
        
        print(f"Directory exists after init: {os.path.exists(directory)}")
        print(f"Directory contents after init: {os.listdir(directory)}")

    async def __aenter__(self):
        return self

    async def __aexit__(self, exc_type, exc_val, exc_tb):
        await self.close()

    async def set_shit(self, key, value) -> None:
        loop = asyncio.get_running_loop()
        
        def set_and_verify():
            print(f"\nSetting key: {key}, value: {value}")
            print(f"Cache size before set: {len(self._cache)}")
            
            # Try the set operation
            result = self._cache.set(key, value)
            print(f"Set operation result: {result}")
            
            # Try to read it back immediately
            read_value = self._cache.get(key)
            print(f"Immediate read back: {read_value}")
            
            print(f"Cache size after set: {len(self._cache)}")
            print(f"All keys after set: {list(self._cache.iterkeys())}")
            
            # Check directory contents
            print(f"Directory contents after set: {os.listdir(self._cache.directory)}")
            
        await loop.run_in_executor(self._pool, set_and_verify)

    async def set_async(self,key, val):
        loop = asyncio.get_running_loop()
        future = loop.run_in_executor(None, self._cache.set, key, val)
        result = await future
        return result
        
    def get_keys(self):
        return list(self._cache.iterkeys())

    async def get(self, key: T) -> U | None:
        loop = asyncio.get_running_loop()
        return await loop.run_in_executor(self._pool, self._cache.get, key)

    async def close(self):
        self._pool.shutdown()
        self._cache.close()

import os

def check_cache_directory(directory):
    print(f"Directory exists: {os.path.exists(directory)}")
    print(f"Directory permissions: {oct(os.stat(directory).st_mode)[-3:]}")
    print(f"Directory contents: {os.listdir(directory)}")

In [105]:
print("\nStarting main:")
cache_dir = "/tmp/batched"

# Check initial state
print(f"Cache directory exists at start: {os.path.exists(cache_dir)}")
if os.path.exists(cache_dir):
    print(f"Initial directory contents: {os.listdir(cache_dir)}")


Starting main:
Cache directory exists at start: True
Initial directory contents: ['cache.db-shm', 'cache.db-wal', 'cache.db']


In [109]:
cache = AsyncDiskCache()


Initializing cache:
Directory exists before init: True
Directory contents before init: ['cache.db-shm', 'cache.db-wal', 'cache.db']
Directory exists after init: True
Directory contents after init: ['cache.db-shm', 'cache.db-wal', 'cache.db']


In [111]:
import asyncio

cache = diskcache.Cache("/tmp/batched", size_limit=100)

async def set_async(key, val):
    loop = asyncio.get_running_loop()
    future = loop.run_in_executor(None, cache.set, key, val)
    result = await future
    return result

asyncio.run(set_async('test-key', 'test-value'))

RuntimeError: asyncio.run() cannot be called from a running event loop

In [88]:
check_cache_directory("/tmp/batched")

Directory exists: True
Directory permissions: 755
Directory contents: ['cache.db-shm', 'cache.db-wal', 'cache.db']


In [107]:
# Set some values
try:
    # Set some values
    await cache.set('key1', 'value1')
    await cache.set('key2', 'value2')
    
    # Add a small delay
    await asyncio.sleep(0.1)
    
    # Final verification
    print("\nFinal verification:")
    print("Direct read key1:", cache._cache.get('key1'))
    print("Direct read key2:", cache._cache.get('key2'))
    print("All keys:", cache.get_keys())
    print("Cache size:", len(cache._cache))
    print("Cache volume:", cache._cache.volume())
    print(f"Final directory contents: {os.listdir(cache_dir)}")
    
    # Check SQLite database content
    #print("\nDatabase contents:")
    #with cache._cache._sql as sql:
    #    cursor = sql.execute('SELECT * FROM Cache')
    #    rows = cursor.fetchall()
    #    print("Raw Cache table contents:", rows)
    
finally:
    cache._cache.close()
    cache._pool.shutdown()


Setting key: key1, value: value1
Cache size before set: 0
Set operation result: True
Immediate read back: None
Cache size after set: 0
All keys after set: []
Directory contents after set: ['cache.db-shm', 'cache.db-wal', 'cache.db']

Setting key: key2, value: value2
Cache size before set: 0
Set operation result: True
Immediate read back: None
Cache size after set: 0
All keys after set: []
Directory contents after set: ['cache.db-shm', 'cache.db-wal', 'cache.db']

Final verification:
Direct read key1: None
Direct read key2: None
All keys: []
Cache size: 0
Cache volume: 32768
Final directory contents: ['cache.db-shm', 'cache.db-wal', 'cache.db']


In [97]:
print(diskcache.__version__)

5.6.3


In [93]:
import sqlite3

def check_db():
    conn = sqlite3.connect('/tmp/batched/cache.db')
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print("Tables in DB:", tables)
    for table in tables:
        cursor.execute(f"SELECT * FROM {table[0]}")
        rows = cursor.fetchall()
        print(f"Contents of {table[0]}:", rows)
    conn.close()

# Add this to main():
check_db()

Tables in DB: [('Settings',), ('Cache',)]
Contents of Settings: [('count', 0), ('size', 0), ('hits', 0), ('misses', 0), ('statistics', 0), ('tag_index', 0), ('eviction_policy', 'least-recently-stored'), ('size_limit', 100), ('cull_limit', 10), ('sqlite_auto_vacuum', 1), ('sqlite_cache_size', 8192), ('sqlite_journal_mode', 'wal'), ('sqlite_mmap_size', 67108864), ('sqlite_synchronous', 1), ('disk_min_file_size', 32768), ('disk_pickle_protocol', 5), ('sqlite_pragma_synchronous', 'FULL'), ('sqlite_pragma_journal_mode', 'WAL')]
Contents of Cache: []


In [77]:
x = await cache.get('key1')
x

In [22]:
cache._cache.volume(), cache._cache.size_limit

(32768, 100)

In [27]:
list(cache._cache.iterkeys())

[]

In [28]:
print(len(cache._cache))  # Check size
print(cache._cache.volume())  # Check volume
print(os.listdir(cache._cache.directory))  # Check actual files

0
32768
['cache.db-shm', 'cache.db-wal', 'cache.db']


In [23]:
await cache.set('FDP', 1)

Setting cache


In [1]:
from batched.utils import AsyncMemoryCache, AsyncDiskCache

In [12]:
cache = AsyncMemoryCache(statistics=True)
#cache = AsyncDiskCache(statistics=False)

In [13]:
from sentence_transformers import SentenceTransformer
import numpy as np
import batched

class SentenceEmbedder:
   def __init__(self, model_name='mixedbread-ai/mxbai-embed-xsmall-v1'):
      self.model = SentenceTransformer(model_name)

   @batched.aio.dynamically(cache=cache)
   def embed_sentences(self, sentences: list[str]) -> list[np.ndarray]:
      # Convert sentences to embeddings
      return self.model.encode(sentences)

# Create an instance of SentenceEmbedder
embedder = SentenceEmbedder()

# Embed single sentences
single_sent = "This is a test sentence."
embedding = embedder.embed_sentences(single_sent)
#awaited_embedding = await embedder.embed_sentences.acall(single_sent)

# Embed a batch of 1000 sentences
batch_sentences = [f"This is test sentence number {i}." for i in range(1000)]
#batch_embeddings = embedder.embed_sentences(batch_sentences)
#awaited_batch_embeddings = await embedder.embed_sentences.acall(batch_sentences)

# Check the statistics
#stats = embedder.embed_sentences.stats

/var/folders/f9/xpzn6dr11glgsc74zx9txsv80000gn/T/ipykernel_3451/1190588346.py:19: RuntimeWarning: coroutine 'AsyncBatchProcessor.__call__' was never awaited
  embedding = embedder.embed_sentences(single_sent)


In [21]:
import tracemalloc
tracemalloc.start()
batch_sentences = [f"This is test sentence number 11." for i in range(640)]
await embedder.embed_sentences(batch_sentences)
print(embedder.embed_sentences.stats)

We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a hit boyz
We got a h

In [9]:
embedder.embed_sentences.stats

BatchProcessorStats(queue_size=0, total_processed=640, total_batches=20, avg_batch_size=32.0, avg_processing_time=0.028366862549819412)

In [11]:
cache._cache.volume(), list(cache._cache.iterkeys()), cache._stats #cache._cache.stats(enable=True)

(32768, [], (0, 6784))

In [22]:
cache._stats

BatchProcessorCacheStats(total_gets=0, total_sets=0, total_hits=0, hit_rate=0.0, utilization_rate=0.0001, total_pops=0, eviction_rate=0.0, total_get_hit_time=0.0, total_get_nonhit_time=0.0, total_set_time=0.0, latency_reduction=0.0)

In [20]:
cache.stats(
    embedder.embed_sentences.stats, reset=True, enable=False
)

BatchProcessorCacheStats(total_gets=0, total_sets=0, total_hits=0, hit_rate=0.0, utilization_rate=0.0001, total_pops=0, eviction_rate=0.0, total_get_hit_time=0.0, total_get_nonhit_time=0.0, total_set_time=0.0, latency_reduction=0.0)

In [10]:
cache._stats

BatchProcessorCacheStats(total_gets=192, total_sets=128, total_hits=64, hit_rate=0.3333333333333333, utilization_rate=0.0064, total_pops=0, eviction_rate=0.0, total_get_hit_time=0.00013303756713867188, total_get_nonhit_time=0.00012540817260742188, total_set_time=0.0002536773681640625, latency_reduction=0.0)

In [ ]:
# hit_rate, access_time/miss_penalty, eviction_rate (kinda related to the hit_rate, if high make cache larger), cache_utilization (if low make cache smaller), load_time (if high make cache smaller), latency_reduction (what is the ultimate cache gain?), mem_overhead